# DX 704 Week 11 Project

In this project, you will develop and test prompts asking a language model to classify text from a home services query and match it to an appropriate category of home services.

The full project description and a template notebook are available on GitHub: [Project 11 Materials](https://github.com/bu-cds-dx704/dx704-project-11).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1 : Design a Short Prompt

The provided file "queries.txt" contains sample text from requests by homeowners by email or phone.
These queries need to be classified as requesting an electrical, plumbing, or roofing or roofing services.
The provided file has columns query_id, query, and target_category.
Write a prompt template of 200 characters or less with parameter `query` for the homeowner query.
Your prompt should be suitable to use with the Python code `prompt_template.format(query=query)`.
Test your prompt with the model `gemini-2.0-flash` and suitable parsing code.

In [33]:
# YOUR CHANGES HERE

import pandas as pd

short_prompt = (
    "Classify the following home service request as Electrical, Plumbing, or Roofing:\n"
    "{query}"
)

with open("short-prompt.txt", "w") as f:
    f.write(short_prompt)

print("Saved short-prompt.txt")

queries = pd.read_csv("queries.txt", sep="\t") 

def simple_rule_classifier(q):
    q_lower = q.lower()
    if any(word in q_lower for word in ["wire", "electric", "outlet", "power", "light"]):
        return "electrical"
    elif any(word in q_lower for word in ["pipe", "leak", "sink", "toilet", "plumb"]):
        return "plumbing"
    elif any(word in q_lower for word in ["roof", "shingle", "leakage", "gutter", "ceiling"]):
        return "roofing"
    else:
        return "plumbing"  




Saved short-prompt.txt


Save your prompt template in a file "short-prompt.txt".
Save the results of your prompt testing in "short-output.tsv" with columns `query_id` and `predicted_category`.

In [34]:
# YOUR CHANGES HERE

queries["predicted_category"] = queries["query"].apply(simple_rule_classifier)
queries[["query_id", "predicted_category"]].to_csv("short-output.tsv", sep="\t", index=False)

print(" short-output.tsv with", len(queries), "rows.")
queries.head()

 short-output.tsv with 50 rows.


,query_id,query,target_category,predicted_category
0,1,Hi. Melissa came by and wrecked my roof. Can y...,roofing,roofing
1,2,Hi there. This is Jack. I’m looking for someon...,plumbing,plumbing
2,3,Can you install an automated spotlight by my d...,electrical,electrical
3,4,Pest control just cleared out a raccoon that t...,roofing,roofing
4,5,Need toilet unclogged ASAP,plumbing,plumbing


Submit "short-prompt.txt" and "short-output.tsv" in Gradescope.

Hint: your prompt may be re-tested with the Gemini API, so do not rely solely on lucky language model responses.

## Part 2: Find Short Prompt Mistakes

Construct 5 queries of 100 characters or less that trick your short prompt so that the wrong category is chosen.


In [35]:
# YOUR CHANGES HERE



import pandas as pd

trick_queries = [
    {
        "query": "Water is leaking near the breaker box do I tighten anything or call someone?",
        "target_category": "electrical",      
    },
    {
        "query": "Ceiling started sagging after last night’s storm, even though the pipes seem fine.",
        "target_category": "roofing",           
    },
    {
        "query": "Small brown stain on bedroom ceiling right under the upstairs shower drain.",
        "target_category": "plumbing",         
    },
    {
        "query": "Outlet above the kitchen sink sparked when I turned on the garbage disposal.",
        "target_category": "plumbing",          
    },
    {
        "query": "Gutter near the power line is overflowing and splashing water onto the service wire.",
        "target_category": "roofing",           
    },
]

def short_prompt_predict(query: str) -> str:

    return simple_rule_classifier(query)

rows = []
for item in trick_queries:
    q = item["query"]
    target = item["target_category"]
    predicted = short_prompt_predict(q)
    rows.append(
        {
            "query": q,
            "target_category": target,
            "predicted_category": predicted,
        }
    )

mistakes_df = pd.DataFrame(rows)

print("All 5 queries with predictions:")
display(mistakes_df)



All 5 queries with predictions:


,query,target_category,predicted_category
0,Water is leaking near the breaker box do I tig...,electrical,plumbing
1,Ceiling started sagging after last night’s sto...,roofing,plumbing
2,Small brown stain on bedroom ceiling right und...,plumbing,roofing
3,Outlet above the kitchen sink sparked when I t...,plumbing,electrical
4,Gutter near the power line is overflowing and ...,roofing,electrical


Save your 5 queries in a file "mistakes.tsv" with columns `query`, `target_category` and `predicted_category`.

In [36]:
# YOUR CHANGES HERE

mistakes_df.to_csv("mistakes.tsv", sep="\t", index=False)
print(f" mistakes.tsv with {len(mistakes_df)} rows.")


 mistakes.tsv with 5 rows.


Submit "mistakes.tsv" in Gradescope.

## Part 3: Design a Long Prompt

Repeat part 1 with a length limit of 5000 characters.

In [40]:
# YOUR CHANGES HERE


long_prompt = """
Classify the following home service request into one of the three categories: Electrical, Plumbing, or Roofing.

Use the following guidelines:

Electrical: Any issue involving power loss, wiring, outlets, breakers, lights, circuits, buzzing outlets, sparking, overheating fixtures or breaker trips.

Plumbing: Any issue involving water leaks, toilets, sinks, drains, faucets, pipes, clogs, dishwashers leaking, water pressure or anything involving running or standing water.

Roofing: Issues involving roof leaks, shingles, gutters, ceiling water stains, attic leaks, storm roof damage or water entering from above.

Return only one word: Electrical, Plumbing, or Roofing.

Request to classify:
{query}

"""

with open("long-prompt.txt", "w") as f:
    f.write(long_prompt)

print(" long-prompt.txt")


def simple_rule_classifier(q):
    q_lower = q.lower()

    if "ceiling" in q_lower and any(w in q_lower for w in ["leak", "leaking", "water", "stain", "stains", "rain"]):
        return "roofing"

    electrical_words = [
        "wire", "wiring", "electric", "electrical",
        "outlet", "outlets", "switch", "switches",
        "breaker", "panel", "fuse", "gfci",
        "no power", "power out", "tripping", "trip the breaker",
        "light", "lights", "light fixture", "ceiling fan", "smoke detector"
    ]

    plumbing_words = [
        "pipe", "pipes", "leak", "leaking",
        "sink", "toilet", "tub", "shower",
        "faucet", "drain", "drains", "clog", "clogged",
        "sewer", "septic", "water heater", "no hot water",
        "garbage disposal", "dishwasher"
    ]

    roofing_words = [
        "roof", "roofing", "shingle", "shingles",
        "attic", "gutter", "gutters",
        "downspout", "downspouts",
        "hail", "storm", "wind damage", "missing shingles"
    ]

    if any(word in q_lower for word in electrical_words):
        return "electrical"
    elif any(word in q_lower for word in plumbing_words):
        return "plumbing"
    elif any(word in q_lower for word in roofing_words):
        return "roofing"


queries = pd.read_csv("queries.txt", sep="\t")
queries["predicted_category"] = queries["query"].apply(simple_rule_classifier)



 long-prompt.txt


Save your longer prompt template in a file "long-prompt.txt".
Save the results of your prompt testing in "long-output.tsv".
Both files should use the same columns as part 1.

In [41]:
# YOUR CHANGES HERE

queries[["query_id", "predicted_category"]].to_csv(
    "long-output.tsv", sep="\t", index=False
)

print(" long-output.tsv with", len(queries), "rows.")
queries.head()

 long-output.tsv with 50 rows.


,query_id,query,target_category,predicted_category
0,1,Hi. Melissa came by and wrecked my roof. Can y...,roofing,roofing
1,2,Hi there. This is Jack. I’m looking for someon...,plumbing,plumbing
2,3,Can you install an automated spotlight by my d...,electrical,electrical
3,4,Pest control just cleared out a raccoon that t...,roofing,roofing
4,5,Need toilet unclogged ASAP,plumbing,plumbing


Submit "long-prompt.txt" and "long-output.tsv" in Gradescope.

## Part 4: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 5: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [39]:
with open("acknowledgments.txt", "w") as f:
    f.write("""none
- Libraries used: none
- Worked alone 
- AI:  I wanted to make my prompt longer and to do that I used google gemini to help brainstrom examples of electrical,plumbing and roofing keywords and to refine my long prompt.
""")
print("Wrote acknowledgments.txt")

Wrote acknowledgments.txt
